In [1]:
import pandas as pd
import numpy as np
import KNN.similarities as sim
import KNN.utils as utils
from tqdm import tqdm

from scipy.spatial.distance import pdist, squareform

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
trainset_df = pd.read_csv('data/experiment_data/ratings.csv')
predicted_df = pd.read_csv('data/experiment_data/predicted_values.csv')

users = list(set(predicted_df['userId']))
functions = ['jaccard', 'euclidean', 'cosine', 'manhattan']
k_list =[1, 2, 3, 5, 10]

In [3]:
def error_rating(user, algorithm, k):
    #print(user)
    # Obtenemos los datos de las películas
    train_movies, test_movies = utils.get_train_test_movies(user)
    all_movies = utils.get_all_movies(user)
    
    # Calculamos la matriz filtrada de similitud
    if algorithm == 'jaccard':
        matrix = pd.read_csv('data/similarity_data/jaccard_matrix.csv')
    if algorithm == 'euclidean':
        matrix = pd.read_csv('data/similarity_data/sim_euclidean_user_{}.csv'.format(user))
    elif algorithm == 'cosine':
        matrix = pd.read_csv('data/similarity_data/sim_cosine_user_{}.csv'.format(user))
    elif algorithm == 'manhattan':
        matrix = pd.read_csv('data/similarity_data/sim_manhattan_user_{}.csv'.format(user))
        
    filtered_matrix = utils.filter_similarity_matrix(matrix, test_movies, train_movies, all_movies)
    return utils.calculate_mean_ratings(filtered_matrix, test_movies, user, k)
    

In [4]:
def average(errors_list):
    return sum(errors_list) / len(errors_list)

In [5]:
errorsDF = pd.DataFrame(columns=functions, index=k_list)

for f in tqdm(range(len(functions))):
    for k in range(len(k_list)):
        avg = []
        for u in range(len(users)):
            avg.append(error_rating(users[u], functions[f], k_list[k]))
        errorsDF[functions[f]][k_list[k]] = average(avg)

100%|██████████| 4/4 [03:56<00:00, 59.00s/it]


In [11]:
errorsDF.transpose()

,1,2,3,5,10
jaccard,0.974077,0.955303,0.954704,0.955156,0.957881
euclidean,0.969043,0.950406,0.95001,0.953391,0.954848
cosine,0.924196,0.932247,0.922775,0.937229,0.951491
manhattan,0.949045,0.961563,0.952259,0.947354,0.947613


In [ ]:
predicted_ratings_DF = pd.read_csv('data/experiment_data/predicted_values.csv')
predicted_ratings_DF['diff'] = predicted_ratings_DF.apply(lambda row: abs(row['rating'] - row['predicted']), axis=1)
predicted_ratings_DF.head()

In [ ]:
print('MAE del NMF:', np.average(predicted_ratings_DF['diff'].values))

In [ ]:
ratings_DF = pd.read_csv('data/experiment_data/ratings.csv')
ratings_DF

In [ ]:
ratings_DF.rating.describe()
